# Example 1 -Atlantic City example

In [ ]:
# Written: sy Aug 2024
# License: BSD-2

"""
 Purpose: Testing Imputer
"""

import os
import sys
import copy
import json

import numpy as np

sys.path.insert(0, "../../")
from brails.utils import Importer
from brails.types.image_set import ImageSet    
from brails.types.asset_inventory import Asset, AssetInventory


In [ ]:
LOCATION_NAME = 'Atlantic City, NJ' #'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'AC.geojson' #'FortMyersInventory_HU.geojson'
NO_POSSIBLE_WORLDS = 1

## Merging NSI and footprint

In [ ]:
importer = Importer()

In [ ]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

In [ ]:
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data(region_boundary_object)

In [ ]:
scraper_class = importer.get_class('USA_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)

In [ ]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    scraper_inventory, "ft", get_extended_features=True)

##  Running imputation

In [ ]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

## Temporarily Assigning features 

In the future, these features should be provided by the brails scraper

In [ ]:
for key, val in imputed_inventory.inventory.items():
    val.add_features({"FloodZone":'V'})

## Run inference

#### Rename features
so that the inferer module can understand

In [ ]:
imputed_inventory.change_feature_names({'erabuilt': 'YearBuilt',
                                        'numstories': 'NumberOfStories',
                                        'basement': 'Basement',
                                        'occupancy': 'OccupancyClass',
                                        'splitlevel': 'SplitLevel',
                                        'found_ht': 'FirstFloorElevation' 
                                       })

#### Run inference

In [ ]:
hurricaneInferer = importer.get_class("HazusInfererFlood")
inferer = hurricaneInferer(input_inventory=imputed_inventory ,clean_features=True,n_possible_worlds=NO_POSSIBLE_WORLDS)
hazus_inferred_inventory = inferer.infer()

#### Check the outcomes

In [ ]:
hazus_inferred_inventory.inventory[0].features

In [ ]:
hazus_inferred_inventory.inventory[312].features